In [ ]:
pip install openai matplotlib pandas seaborn scikit-learn

In [ ]:
import openai
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



# 📥 Đọc dữ liệu từ file JSONL
with open("/opt/workspace/gen_1604_formated.jsonl", "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# 📌 Trích tất cả câu hỏi từ người dùng
questions = [
    m["content"].strip()
    for item in data
    for m in item.get("messages", [])
    if m["role"] == "user"
]

# 🧠 Prompt: yêu cầu đếm tần suất các câu hỏi trùng lặp và trả về câu hỏi đại diện
prompt = f"""
Bạn là trợ lý phân tích và gom nhóm nội dung câu hỏi người dùng.

🎯 Nhiệm vụ:
1. Đọc danh sách các câu hỏi dưới đây.
2. Gom các câu hỏi có nội dung hoặc mục đích tương tự thành **cùng một nhóm chủ đề** (ví dụ: câu hỏi về sự cố, thiết bị, người dùng, lịch bảo dưỡng, quyền truy cập...).
3. Mỗi nhóm cần có:
   - Một **tiêu đề chủ đề khái quát** (ngắn gọn, rõ ràng, ví dụ: "Sự cố", "Người dùng", "Tài sản", v.v.).
   - Một **câu hỏi đại diện** cụ thể nhất trong nhóm đó (ví dụ tiêu biểu).
   - **Số lượng câu hỏi** thuộc nhóm đó (tần suất).
4. Sắp xếp kết quả theo **số lượng câu hỏi giảm dần**.
5. Trình bày theo bảng văn bản. Mỗi dòng gồm:
   `Số lượng | Chủ đề | Câu hỏi đại diện`
6. Không tạo JSON, không tạo danh sách chi tiết từng câu, không mô tả thừa, chỉ liệt kê bảng kết quả cuối cùng.

Dưới đây là danh sách câu hỏi:
{chr(10).join(f"- {q}" for q in questions)}
"""

# 🔗 Gọi OpenAI API
response = openai.ChatCompletion.create(
    model="gpt-4o-mini",
    temperature=0.2,
    messages=[
        {"role": "system", "content": "Bạn là trợ lý chuyên phân tích và thống kê dữ liệu văn bản."},
        {"role": "user", "content": prompt}
    ]
)

# 📋 In kết quả trả về
result_text = response.choices[0].message.content.strip()
print("\n📋 TẦN SUẤT CÁC CÂU HỎI TRÙNG LẶP NHIỀU NHẤT:\n")
print(result_text)
